In [1]:
from services.text_embedding_service import TextEmbeddingService
text_embedding_service = TextEmbeddingService()

2024-06-08 15:38:26.067963: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-08 15:38:26.099193: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-08 15:38:26.444702: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-08 15:38:32.021623: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
from services.dimensionality_reduction_service import DimensionalityReductionService
dimensionality_reduction_service =  DimensionalityReductionService()

/mnt/work/data/umt/semester_7/dlnn/project/recommendation_engine/.venv/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from sklearn.discriminant_analysis import StandardScaler
scaler = StandardScaler()

In [4]:
from typing import Iterable
from numpy import float64
import pandas as pd
from numpy.typing import NDArray

def add_text_embeddings(df: pd.DataFrame, column_key: str):
    embeddings = dimensionality_reduction_service.reduce_dimensions(
        text_embedding_service.embed_sentences(df[column_key]),
        n_dimensions=20,
    )

    df.drop(columns=["text"], inplace=True)
    for i in range(20):
        df[f"dimension_{i}"] = embeddings.T[i]

def add_user_interactions_into_df(df:pd.DataFrame, interactions: Iterable[bool], column_name: str):
    df[column_name] = interactions

def scale(df: pd.DataFrame) -> NDArray[float64]:
    return scaler.fit_transform(df)
    


In [5]:
import pandas as pd

df = pd.read_csv("./data/dev/1_popularity_calculated_posts/posts.csv")
df.head()

,text,popularity
0,Attended a thought-provoking seminar on sustai...,0.845
1,Immersed in the profound wisdom of environment...,0.782
2,Embarked on a journey into the world of enviro...,0.801
3,Reflecting on the power of community-driven co...,0.709
4,Diving deep into the complexities of climate p...,0.932


In [6]:
df_user_interactions = pd.read_csv("./data/dev/2_raw_user_interactions/elias.csv")
df_user_interactions.head()

,text,user_interacted
0,Attended a thought-provoking seminar on sustai...,True
1,Immersed in the profound wisdom of environment...,False
2,Embarked on a journey into the world of enviro...,False
3,Reflecting on the power of community-driven co...,True
4,Diving deep into the complexities of climate p...,True


In [7]:
add_text_embeddings(df, "text")
df.head()

,popularity,dimension_0,dimension_1,dimension_2,dimension_3,dimension_4,dimension_5,dimension_6,dimension_7,dimension_8,...,dimension_10,dimension_11,dimension_12,dimension_13,dimension_14,dimension_15,dimension_16,dimension_17,dimension_18,dimension_19
0,0.845,10.540497,-0.102011,6.758393,8.954913,-0.420535,2.140978,4.482683,2.808676,2.827250,...,4.378349,4.537199,2.778873,7.803953,5.595557,5.271529,3.697397,3.884354,3.626913,2.628710
1,0.782,10.953199,0.558199,6.828146,8.417348,0.550673,1.789602,3.397329,2.656020,3.332409,...,4.918856,3.914625,3.900043,7.142853,6.534964,4.350521,3.587431,3.115053,4.110089,2.228625
2,0.801,10.499941,0.577348,7.139091,8.535638,0.144875,1.962428,3.593105,3.167344,2.840610,...,4.733602,3.821448,3.601897,7.095600,6.386204,4.665680,3.805481,3.486660,4.080857,2.768532
3,0.709,10.513778,0.198391,5.957372,8.524588,0.181251,2.419273,3.686087,2.262808,4.088174,...,4.428041,4.038043,3.996719,7.432343,6.031281,4.869900,3.175696,3.239911,4.067169,2.641047
4,0.932,10.386322,0.499503,6.367846,8.111738,0.442485,1.825599,3.572367,2.295762,3.571672,...,4.778780,4.631165,4.319926,7.530394,6.144840,4.316530,3.078682,3.187951,4.602834,2.832357


In [8]:
add_user_interactions_into_df(df, pd.Series(df_user_interactions["user_interacted"]), "user_interacted")
df.head()

,popularity,dimension_0,dimension_1,dimension_2,dimension_3,dimension_4,dimension_5,dimension_6,dimension_7,dimension_8,...,dimension_11,dimension_12,dimension_13,dimension_14,dimension_15,dimension_16,dimension_17,dimension_18,dimension_19,user_interacted
0,0.845,10.540497,-0.102011,6.758393,8.954913,-0.420535,2.140978,4.482683,2.808676,2.827250,...,4.537199,2.778873,7.803953,5.595557,5.271529,3.697397,3.884354,3.626913,2.628710,True
1,0.782,10.953199,0.558199,6.828146,8.417348,0.550673,1.789602,3.397329,2.656020,3.332409,...,3.914625,3.900043,7.142853,6.534964,4.350521,3.587431,3.115053,4.110089,2.228625,False
2,0.801,10.499941,0.577348,7.139091,8.535638,0.144875,1.962428,3.593105,3.167344,2.840610,...,3.821448,3.601897,7.095600,6.386204,4.665680,3.805481,3.486660,4.080857,2.768532,False
3,0.709,10.513778,0.198391,5.957372,8.524588,0.181251,2.419273,3.686087,2.262808,4.088174,...,4.038043,3.996719,7.432343,6.031281,4.869900,3.175696,3.239911,4.067169,2.641047,True
4,0.932,10.386322,0.499503,6.367846,8.111738,0.442485,1.825599,3.572367,2.295762,3.571672,...,4.631165,4.319926,7.530394,6.144840,4.316530,3.078682,3.187951,4.602834,2.832357,True


In [9]:
df.shape

(600, 22)

In [10]:
df.describe()

,popularity,dimension_0,dimension_1,dimension_2,dimension_3,dimension_4,dimension_5,dimension_6,dimension_7,dimension_8,...,dimension_10,dimension_11,dimension_12,dimension_13,dimension_14,dimension_15,dimension_16,dimension_17,dimension_18,dimension_19
count,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,...,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000
mean,0.730960,10.506225,0.466323,6.464222,8.552615,0.351652,2.136628,3.541617,3.131048,3.430079,...,4.502782,4.025246,4.119323,7.046213,6.464838,4.742248,3.548664,3.398571,4.151422,2.885471
std,0.097188,1.047137,0.950327,0.841987,0.568096,0.627723,0.511682,0.519779,0.708064,0.598422,...,0.497431,0.852157,0.752034,0.604641,0.604622,0.636259,0.486327,0.687271,0.702609,0.429647
min,0.510000,1.768262,-1.117502,2.569502,4.608588,-0.577011,-0.028187,1.943360,1.452003,2.261267,...,1.486258,2.244128,2.497031,4.657429,4.573390,3.374552,2.042167,0.285875,3.234218,1.110565
25%,0.661500,10.425994,-0.232238,6.020015,8.313464,-0.002676,1.845241,3.351331,2.596863,2.908003,...,4.283242,3.531170,3.678332,6.792059,6.142181,4.351744,3.224217,3.010744,3.679342,2.641783
50%,0.721000,10.617772,0.271105,6.527324,8.661933,0.326494,2.139662,3.509350,3.169827,3.453788,...,4.550075,4.031063,4.131632,7.149879,6.377975,4.677324,3.623551,3.402105,4.087723,2.870856
75%,0.798000,10.931001,0.977967,6.952755,8.916829,0.581063,2.385137,3.725767,3.570355,3.981026,...,4.750841,4.358574,4.632129,7.357036,6.708506,5.150249,3.938796,3.751822,4.453119,3.136365
max,0.978000,11.517872,2.926978,8.349368,9.149137,4.681233,4.029579,6.448887,5.576534,4.638673,...,5.754522,9.302798,7.645267,8.350308,8.776631,8.142584,4.448366,5.509140,8.848859,3.952102


In [11]:
from sklearn.model_selection import train_test_split
from numpy import float64


Y: "pd.Series[bool]" = df.pop("user_interacted")
X = scale(df)
splitted_data: list[NDArray[float64]] = train_test_split(
    X,
    Y,
    test_size=0.3,
    random_state=26,
)
X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = splitted_data

In [12]:
from sklearn.neural_network import MLPClassifier
model = MLPClassifier(hidden_layer_sizes=(12, 12, 12, 12), max_iter=600, random_state=26)

In [13]:
model.fit(X_TRAIN, Y_TRAIN)

MLPClassifier(hidden_layer_sizes=(12, 12, 12, 12), max_iter=600,
              random_state=26)

In [14]:
Y_TEST_PRED = model.predict(X_TEST)

In [15]:
from numpy import sqrt
from sklearn.metrics import confusion_matrix


conf_mat = confusion_matrix(Y_TEST, Y_TEST_PRED)

TN, FP, FN, TP = conf_mat.ravel()

accuracy = (TP + TN) / (TP + FP + TN + FN)
recall = TP / (TP + FN)
specificity = TN / (FP + TN)
precision = TP / (TP + FP)
mcc = ((TP * TN) - (FP * FN)) / sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN))
f1_score_ = (2 * precision * recall) / (precision + recall)

print('MODEL ASSESSMENT RF')
print('\nConfusion Matrix\n', conf_mat, end='\n\n')
print('            Accuracy : ', accuracy)
print('Recall / Sensitivity : ', recall)
print('         Specificity : ', specificity)
print('           Precision : ', precision)
print('                 MCC : ', mcc)
print('            F1-Score : ', f1_score_)

MODEL ASSESSMENT RF

Confusion Matrix
 [[79 22]
 [29 50]]

            Accuracy :  0.7166666666666667
Recall / Sensitivity :  0.6329113924050633
         Specificity :  0.7821782178217822
           Precision :  0.6944444444444444
                 MCC :  0.4204728606915087
            F1-Score :  0.662251655629139
